# **Click-Through Rate Prediction**
#### This exercise covers the steps for creating a click-through rate (CTR) prediction pipeline. 
#### ** This exercise will cover: **
+  ####*Part 1:* Parse CTR data and generate OHE features
+  ####*Part 2:* Training with LogisticRegression and logloss evaluation
 
#### Note that, for reference, you can look up the details of the relevant Spark methods in [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) and the relevant NumPy methods in the [NumPy Reference](http://docs.scipy.org/doc/numpy/reference/index.html)

### **Part 1: Parse CTR data and generate OHE features**

#### **Data loading and visualizing the raw data**

In [ ]:
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('HandsOnML', 'dac_sample.txt')
fileName = os.path.join(baseDir, inputPath)

if os.path.isfile(fileName):
    rawData = (sc
               .textFile(fileName, 2)
               .map(lambda x: x.replace('\t', ',')))# work with either ',' or '\t' separated data
    print rawData.take(1)

#### **Splitting the data **
####  1- Splitting the data into training, validation, and test sets using the [randomSplit method](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) with the specified weights and seed to create RDDs storing each of these datasets.
#### 2 - [Cache](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) each of these RDDs. 
#### 3- Compute the size of each dataset.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
weights = [.8, .1, .1]
seed = None
# Use randomSplit with weights and seed
rawTrainData, rawValidationData, rawTestData = <FILL IN>
# Cache the data because we need to use these datasets later
<FILL IN>
<FILL IN>
<FILL IN>

nTrain = <FILL IN>
nVal = <FILL IN>
nTest = <FILL IN>
print nTrain, nVal, nTest, nTrain + nVal + nTest
print rawData.take(1)

#### ** Creating ONE Dictionary**
#### Generating a dictionary containing a list of "distinct (featureID, value) : Unique Integer" from the raw training data. We will ignore the first field (which is the 0-1 label), and parse the remaining fields (or raw features). Note : In our exercise, on apply the OneHot Encoding on all the features.

In [ ]:
#Fist we should get a RDD of a list of (featureID,value) from the raw training data
# TODO: Replace <FILL IN> with appropriate code
def parseData(data):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        data (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    
    data_chain = <FILL IN>
    
    fv_tuple = <FILL IN>
    
    return fv_tuple
    
parsedTrainFeat = rawTrainData.map(parseData)
print parsedTrainFeat.take(1)

In [ ]:
# Create an OHE dictionary from the RDD of the list of (featureID, value)
# generated in the previous function 

In [ ]:
##### OHE dictionary sample : 
#cityOHEDict[(0,'Beijing')] = 0
#cityOHEDict[(0,'Paris')] = 1
#cityOHEDict[(0,'London')] = 2
#cityOHEDict[(0,'New York')] = 3
#cityOHEDict[(1, 'Asia')] = 4
#cityOHEDict[(1, 'Europe')] = 5
#cityOHEDict[(1, 'American')] = 6
#cityOHEDict[(2, 'very much')] = 7
#cityOHEDict[(2, 'a little')] = 8

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
def createOneHotDictionary(inputData):
    """Creates a one-hot-encoder dictionary based on the input data.
    
    Hint : function zipWithIndex can be useful

    Args:
        inputData (RDD of lists of (int, str)): An RDD of observations where each observation is
            made up of a list of (featureID, value) tuples.

    Returns:
        dictionary: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    
    DistinctFeatures = <FILL IN>
    
    OHEDictionary = <FILL IN>
    
    return OHEDictionary

OHEDictionary = createOneHotDictionary(parsedTrainFeat)
print OHEDictionary.items()[0]

numCtrOHEFeats = len(OHEDictionary.keys())
print numCtrOHEFeats

#### ** Extracting the features using OHE dictionary and Transforming training/validation/test data to a RDD of LabeledPoint **

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(parsedFeats, OHEDictionary):
    """Produce a OneHot encoding from 
         1- a list of features parsed from raw data
         2- an OneHot dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.
        !Important!:Because some categorical values will likely appear in validation/Test data 
        that did not exist in the training data. 
        To deal with this situation, ignoring unseen categories in validation/test data

    Args:
        parsedFeats (list of (int, str)): The features corresponding to a single observation.Each
            feature consists of a tuple of featureID and the feature's value.
        OHEDictionary: A mapping of (featureID, value) to unique integer.

    Returns:
        SparseVector: A SparseVector of length numOHEFeats with indicies equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    return <FILL IN>

def getLabeledPoint(data, OHEDictionary):
    """Obtain the label and feature vector for this raw observation.

    Note:
        You must use the function `oneHotEncoding` in this implementation.
    Args:
        data (str): A comma separated string where the first value is the label and the rest
            are features.
        OHEDictionary (dictionary of (int, str) to int): Mapping of (featureID, value) to unique integer.

    Returns:
        LabeledPoint: Contains the label for the observation and the one-hot-encoding of the
            raw features based on the provided OHE dictionary.
    """
    
    parsedFeat = <FILL IN>
    
    sparseVector = <FILL IN>
    
    return <FILL IN>

OHETrainData = rawTrainData.map(lambda data: getLabeledPoint(data, OHEDictionary))
OHETrainData.cache()
print OHETrainData.take(1)

OHEValidationData = rawValidationData.map(lambda data: getLabeledPoint(data, OHEDictionary))
OHEValidationData.cache()
print OHEValidationData.take(1)

OHETestData = rawValidationData.map(lambda data: getLabeledPoint(data, OHEDictionary))
OHETestData.cache()
print OHETestData.take(1)

### ** Part 2: CTR prediction and logloss evaluation **

#### ** Log loss **
#### Throughout this exercise, we will use log loss to evaluate the quality of models.  Log loss is defined as: $$  \begin{align} \scriptsize \ell_{log}(p, y) = \begin{cases} -\log (p) & \text{if } y = 1 \\\ -\log(1-p) & \text{if } y = 0 \end{cases} \end{align} $$ where $ \scriptsize p$ is a probability between 0 and 1 and $ \scriptsize y$ is a label of either 0 or 1.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from math import log

def computeLogLoss(p, y):
    """Calculates the value of log loss for a given probabilty and label.

    Note:
        log(0) is undefined, so when p is 0 we need to add a small value (epsilon) to it
        and when p is 1 we need to subtract a small value (epsilon) from it.

    Args:
        p (float): A probabilty between 0 and 1.
        y (int): A label.  Takes on the values 0 and 1.

    Returns:
        float: The log loss value.
    """
    epsilon = 10e-12
    
    if y == 1.:
        return <FILL IN>
    if y == 0.:
        return <FILL IN>

#### ** Predicted probability **
#### In order to compute the log loss for the model we train, we need to write code to generate predictions from the model. Write a function that computes the raw linear prediction (t = x.dot(w) + intercept) from this logistic regression model and then passes it through a [sigmoid function](http://en.wikipedia.org/wiki/Sigmoid_function) $ \scriptsize \sigma(t) = (1+ e^{-t})^{-1} $ to return the model's probabilistic prediction. Then compute probabilistic predictions on the training data.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from math import exp #  exp(-t) = e^-t

def getProbability(x, w, intercept):
    """Calculate the probability for an observation given a set of weights and intercept.

    Note:
        We'll bound our raw prediction between 20 and -20 for numerical purposes.

    Args:
        x (SparseVector): A vector with values of 1.0 for features that exist in this
            observation and 0.0 otherwise.
        w (DenseVector): A vector of weights (betas) for the model.
        intercept (float): The model's bias.

    Returns:
        float: A probability between 0 and 1.
    """
    rawPrediction = <FILL IN>

    # Bound the raw prediction value
    
    rawPrediction = min(rawPrediction, 20)
    rawPrediction = max(rawPrediction, -20)
    
    return <FILL IN>

#### ** Evaluate the model **
#### Now we write a general function that takes as input a model and data, and outputs the log loss. 

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
def computeLogLossOfModel(model, data):
    """Calculates the log loss for the data given the model.

    Args:
        model (LogisticRegressionModel): A trained logistic regression model.
        data (RDD of LabeledPoint): Labels and features for each observation.

    Returns:
        float: Log loss for the data.
    """
    
    probability_label_tuple = <FILL IN>
    
    loss = <FILL IN>
    
    return loss.mean()

#### ** Traning the model with Logistic regression **
####  First use [LogisticRegressionWithSGD](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithSGD) to train serveral models using `OHETrainData` with the given differente hyperparameter configuration. Getting the best model using validation dataset by calculating Log Loss with the parameters of every model

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

numIters = 80
regType = 'l2'
includeIntercept = True

# Initialize variables
bestModel = None
bestLogLoss = float('inf')

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
stepSizes = [1,10]
regParams = [1e-6,1e-3]
for stepSize in stepSizes:
    for regParam in regParams:
        model = <FILL IN>
        #calculate the log loss on validation dataset using the trained model
        logLossValidation = <FILL IN>
        print ('\tstepSize = {0:.1f}, regParam = {1:.0e}: logloss = {2:.3f}'
               .format(stepSize, regParam, logLossVa))
        if (logLossValidation < bestLogLoss):
            bestModel = model
            bestLogLoss = logLossVa

#### ** Evaluate on the test set **
#### Finally, do the predictions with the best model on the test set and calcule the accuracy of prediction.

In [ ]:
#from pyspark.mllib.evaluation import BinaryClassificationMetrics from spark 1.4
# TODO: Replace <FILL IN> with appropriate code

# doing the prediction on the test dataset using the best model
predictionAndLabel = <FILL IN>
print predictionAndLabel.take(1)

# calculate the accuracy of the predicton of the best model in this exercise
accuracy = <FILL IN>
print ('The accuracy on Test Dataset with the best model:\n\taccuracy = {0:.3f}'
       .format(accuracy))

#from spark 1.4
#metrics = BinaryClassificationMetrics(predictionAndLabel) 
#print metrics.areaUnderROC()

sortedWeights = sorted(bestModel.weights)
print sortedWeights[:5], bestModel.intercept

In [ ]:
# You can go further using the hashing features for this exercise. Good Luck